In [1]:
import json
from itertools import chain
from math import ceil

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

/home/ubuntu/anaconda2/envs/fashion/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

datagen = ImageDataGenerator()
gen = datagen.flow_from_directory('images100',
                                  target_size=(224, 224),
                                  class_mode=None, 
                                  shuffle=False,
                                  batch_size=32)

Found 100 images belonging to 1 classes.


In [3]:
num_images = len(gen.filenames)
num_images

100

In [4]:
batches = []
for i in range(ceil(num_images / gen.batch_size)):
    batches.append(gen.next())
    
img_arrays = np.array(list(chain(*batches)))
# img_arrays = np.array(list(chain.from_iterable(batches)))

del batches

In [5]:
img_arrays.shape

(100, 224, 224, 3)

In [6]:
img_ids = np.array([int(fn[6:-4]) for fn in gen.filenames])
img_ids.shape

(100,)

# id-to-labels mapping

In [7]:
with open('data/train.json') as fp:
    train = json.load(fp)
    
df = pd.DataFrame(train['annotations'][:100000])
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df['labelId']),
                          columns=mlb.classes_,
                          index=df.index))
df = df.drop('labelId', axis=1)
df.imageId = df.imageId.astype(int)
df = df.set_index('imageId')
df.head()

,1,10,100,101,102,103,104,105,106,107,...,90,91,92,93,94,95,96,97,98,99
imageId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.shape

(100000, 227)

In [9]:
img_labels = df.loc[img_ids]
img_labels.shape

(100, 227)

## arrays size in MB

In [10]:
img_arrays.size * img_arrays.itemsize / 1024**2

57.421875